In [80]:
def write_answer(ans, num):
    file_name = "answer_" + str(num) + ".txt"
    with open(file_name, "w") as fout:
        fout.write(str(ans))

# Задание: Градиентный бустинг своими руками

В этом задании будет использоваться датасет boston из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

### Задание 1

Как вы уже знаете из лекций, бустинг - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки. Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь L - квадрат отклонения ответа композиции a(x) от правильного ответа y на данном x.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [3]:
import numpy as np
import pandas as pd

In [23]:
from sklearn import datasets
boston = datasets.load_boston()
X = boston.data
y = boston.target

In [31]:
X.shape

(506, 13)

In [37]:
X_train, X_test = X[:int(.75*len(X)),:], X[int(.75*len(X)):,:]
y_train, y_test = y[:int(.75*len(y))], y[int(.75*len(y)):]

In [66]:
def s(X, y, a_prev):
    answers = map(a_prev, X)
    return -(np.array(y) - answers)

### Задание 2

Заведите массив для объектов DecisionTreeRegressor (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). В цикле обучите последовательно 50 решающих деревьев с параметрами max_depth=5 и random_state=42 (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке X:

### def gbm_predict(X):
### return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью mean_squared_error в sklearn.metrics. Возведите результат в степень 0.5, чтобы получить RMSE. Полученное значение RMSE — ответ в пункте 2.

In [67]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] 
                 for algo, coeff in zip(base_algorithms_list, coefficients_list)
                ]) 
            for x in X
           ]

In [78]:
from sklearn.tree import DecisionTreeRegressor as dtr

base_algorithms_list = []
coefficients_list = []

for i in range(50):
    estimator = dtr(max_depth=5, random_state=42)
    estimator.fit(X_train, 
                  np.array(y_train) - gbm_predict(X_train))
    base_algorithms_list.append(estimator)
    coefficients_list.append(.9)

In [83]:
from sklearn.metrics import mean_squared_error as mse

rmse = lambda y_true, y_pred: mse(y_true, y_pred)**.5

In [82]:
y_test_pred = gbm_predict(X_test)
res = rmse(y_test, y_test_pred)
write_answer(res, 2)
print res

5.47665097417


### Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле 0.9 / (1.0 + i), где i - номер итерации (от 0 до 49). Используйте качество работы алгоритма как ответ в пункте 3. В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [84]:
base_algorithms_list = []
coefficients_list = []

for i in range(50):
    estimator = dtr(max_depth=5, random_state=42)
    estimator.fit(X_train, 
                  np.array(y_train) - gbm_predict(X_train))
    base_algorithms_list.append(estimator)
    coefficients_list.append(.9/(1.+i*1.))

In [85]:
y_test_pred = gbm_predict(X_test)
res = rmse(y_test, y_test_pred)
write_answer(res, 3)
print res

4.81089328026


### Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке sklearn, так и в сторонней библиотеке XGBoost, которая имеет свой питоновский интерфейс. На практике XGBoost работает заметно лучше GradientBoostingRegressor из sklearn, но для этого задания вы можете использовать любую реализацию. Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет ответ в п.4):

- 1) С увеличением числа деревьев начиная с некоторого момента качество работы градиентного бустинга не меняется существенно.

+ 2) С увеличением числа деревьев начиная с некоторого момента градиентный бустинг начинает переобучаться.

+ 3) С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться

- 4) С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [86]:
import xgboost as xgb

In [ ]:
# xgb.XGBRegressor(self,
#                  max_depth=3, 
#                  learning_rate=0.1, 
#                  n_estimators=100, 
#                  silent=True, 
#                  objective='reg:linear', 
#                  nthread=-1, 
#                  gamma=0, 
#                  min_child_weight=1, 
#                  max_delta_step=0, 
#                  subsample=1, 
#                  colsample_bytree=1, 
#                  base_score=0.5, 
#                  seed=0, 
#                  missing=None)

In [101]:
for n_estimators in range(1, 6) + range(5, 51, 5) + range(100, 1001, 100):
    estimator = xgb.XGBRegressor(n_estimators=n_estimators)
    estimator.fit(X_train, y_train)
    prediction = estimator.predict(X_test)
    print 'n_estimators=', n_estimators,': rmse=', rmse(y_test, prediction)

n_estimators= 1 : rmse= 13.6612597946
n_estimators= 2 : rmse= 12.1286595112
n_estimators= 3 : rmse= 10.8374001345
n_estimators= 4 : rmse= 9.64173545066
n_estimators= 5 : rmse= 8.64072274593
n_estimators= 5 : rmse= 8.64072274593
n_estimators= 10 : rmse= 5.18556091032
n_estimators= 15 : rmse= 4.2249503942
n_estimators= 20 : rmse= 4.34007132038
n_estimators= 25 : rmse= 4.63856762128
n_estimators= 30 : rmse= 4.94223470993
n_estimators= 35 : rmse= 5.11121041716
n_estimators= 40 : rmse= 5.19724487413
n_estimators= 45 : rmse= 5.15179217693
n_estimators= 50 : rmse= 5.13507312859
n_estimators= 100 : rmse= 4.78117872688
n_estimators= 200 : rmse= 4.65872563511
n_estimators= 300 : rmse= 4.59420527161
n_estimators= 400 : rmse= 4.5917611326
n_estimators= 500 : rmse= 4.58394595489
n_estimators= 600 : rmse= 4.5890706904
n_estimators= 700 : rmse= 4.58291334041
n_estimators= 800 : rmse= 4.58701010165
n_estimators= 900 : rmse= 4.58965765397
n_estimators= 1000 : rmse= 4.59599019849


In [109]:
for max_depth in range(1, 5) + range(5, 36, 2):
    estimator = xgb.XGBRegressor(n_estimators=100,
                                 max_depth=max_depth)
    estimator.fit(X_train, y_train)
    prediction = estimator.predict(X_test)
    print 'max_depth=', max_depth,': rmse=', rmse(y_test, prediction)


max_depth= 1 : rmse= 5.84567448655
max_depth= 2 : rmse= 5.19298446582
max_depth= 3 : rmse= 4.78117872688
max_depth= 4 : rmse= 4.92832991345
max_depth= 5 : rmse= 4.88285430877
max_depth= 7 : rmse= 5.3037654803
max_depth= 9 : rmse= 5.16297460846
max_depth= 11 : rmse= 5.02039537115
max_depth= 13 : rmse= 5.13785617497
max_depth= 15 : rmse= 5.21681721538
max_depth= 17 : rmse= 5.16948301337
max_depth= 19 : rmse= 5.17403069937
max_depth= 21 : rmse= 5.18211505173
max_depth= 23 : rmse= 5.18211505173
max_depth= 25 : rmse= 5.18211505173
max_depth= 27 : rmse= 5.18211505173
max_depth= 29 : rmse= 5.18211505173
max_depth= 31 : rmse= 5.18211505173
max_depth= 33 : rmse= 5.18211505173
max_depth= 35 : rmse= 5.18211505173


In [114]:
file_name = "answer_4.txt"
with open(file_name, "w") as fout:
        fout.write("2 3")

### Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. Для этого обучите LinearRegression из sklearn.linear_model (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке RMSE. Полученное качество - ответ в пункте 5. В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [112]:
from sklearn.linear_model import LinearRegression

estimator = LinearRegression()
estimator.fit(X_train, y_train)
prediction = estimator.predict(X_test)
ans = rmse(y_test, prediction)
write_answer(ans, 5)
print ans

8.27046803494
